In [1]:
import sys
sys.path.append("/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs")

import numpy as np 
import pandas as pd 
from scipy import integrate
from tools import constants

In [2]:
# User defined inputs

galaxy_name = "m12i_res7100_md"
galaxy_type = "zoom_in"
redshift = "0.0"

# galaxy_name = "gal50"
# galaxy_type = "firebox"
# redshift = "0.0"

wavelength_min = 8     # um
wavelength_max = 1000  # um



In [3]:
directory_name = "voronoi_1e6"
FILE_DIR = f"/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}"

sed = np.loadtxt(
    fname=f"{FILE_DIR}/{galaxy_name}_i0_sed.dat"
)

indices = np.where(
    (sed[:,0] >= wavelength_min) &
    (sed[:,0] <= wavelength_max)
)

# Use only the wavelengths of interest
sed = sed[indices]

##### Calculate luminosity
# Integrate 
integrated_total_flux = integrate.simpson(
    y = sed[:,1],
    x = sed[:,0],
) # W/m^2

# Multiply with an area
# Distance between the galaxy and detector is 10Mpc
luminosity = integrated_total_flux * (4 * np.pi * (10*constants.Mpc2meter)**2) * constants.w2ergs # erg/s


luminosity

8.056766174160508e+43

In [4]:
%%time
# Luminosity expected to receive from the stars: 

columns = [
    "x",
    "y",
    "z",
    "vx",
    "vy",
    "vz",
    "metallicity",
    "mass",
    "age"
]

stars = pd.DataFrame(
    np.loadtxt(fname = f"{FILE_DIR}/comprehensive_star.txt"), 
    columns=columns
)


CPU times: user 6.48 s, sys: 373 ms, total: 6.85 s
Wall time: 7.1 s


In [5]:
Lsolar2ergs = 3.826e33 # ergs/s

stefan_boltzmann_sigma = 5.67e-5 # erg cm^-2 T^-4 s^-1 
Tstar = 5600 # K 
sun_radius = 696340e5 # cm

flux_single_star = stefan_boltzmann_sigma * (Tstar**4)

number_of_stars = sum(stars["mass"]) 
print(f"number_of_stars: {number_of_stars/1e10}e10")

Lstars =  flux_single_star * ( 4 * np.pi * sun_radius**2 ) * number_of_stars

Lstars

number_of_stars: 3.508284297354603e10


1.1920171309513535e+44

In [6]:
Lsolar2ergs*number_of_stars

1.3422695721678711e+44

In [9]:
%%time
gas = np.loadtxt(fname = f"{FILE_DIR}/comprehensive_gas.txt")

len(gas)

CPU times: user 1.59 s, sys: 79 ms, total: 1.67 s
Wall time: 1.67 s


96028